In [11]:
import dask.dataframe as dd
from dask import delayed
import boto3
import pandas as pd

In [15]:
from dask.distributed import Client, progress
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36163 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 18.68 GiB


In [18]:
client.close()

In [16]:
s3 = boto3.client("s3")

bucket = 'rec-gov-study'

In [17]:
%%time

#define s3 location keys
Campsites_API_file = 'data/Campsites_API_v1.csv'
fy20_historical_reservations_file = 'data/fy20_historical_reservations_full.csv'

#create df from reservation data on s3 bucket. 
#this is a large file and may take a few minutes
boto_object = s3.get_object(Bucket=bucket, Key=fy20_historical_reservations_file)
date_cols = ['startdate', 'enddate', 'orderdate']
df_all_res = pd.read_csv(boto_object['Body'], parse_dates=date_cols, nrows=100000)

print('dowloaded all reservation records from s3')

#create df from campsite api data on s3 bucket.
boto_object = s3.get_object(Bucket=bucket, Key=Campsites_API_file)
df_campsite_api = pd.read_csv(boto_object['Body'])

print('dowloaded campsite api from s3')

#coerce dates for endate
df_all_res['enddate'] = pd.to_datetime(df_all_res['enddate'], errors='coerce')

KeyboardInterrupt: 

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
#looking at the campsite api, how many campsites are at each facility
df_facility_max = df_campsite_api.groupby('FacilityID')['CampsiteID'].count().reset_index()
df_facility_max.columns = ['facilityid', 'total_num_campsites']

print('created new facility max dataframe')


#Identify what reservations are campsite related based on facility ID
df_merge_all_data = pd.merge(df_all_res, df_facility_max, on='facilityid', how='left')
df_merge_all_data['campsite'] = ~df_merge_all_data['total_num_campsites'].isnull()

print('merged dataframes')

ddf = dd.from_pandas(df_merge_all_data, npartitions=4)
print('transformed into dask dataframe')
